In [1]:
import tabula
import pandas as pd
import numpy as np
import os
import pdfplumber
from datetime import datetime

In [2]:
df_aun_mas_completo = tabula.read_pdf('00959791_JEAN PAOLO PARDO JARA_FQ.pdf', columns = [64, 218, 264, 296, 346, 373, 424, 450, 507], pages = 1, guess = False)[0]

In [3]:
def obtener_entradas_todos_los_pacientes():
    entradas_todos_los_pacientes = []

    for nombre_archivo in os.listdir():
        if '.pdf' in nombre_archivo:
            datos_demograficos = obtener_datos_demograficos_de_un_paciente(nombre_archivo)
            antibiogramas = obtener_antibiogramas(nombre_archivo)
            microorganismos = obtener_microorganismos_paciente(nombre_archivo)

            entradas_todos_los_pacientes.append(antibiograma)

    return entradas_todos_los_pacientes

In [10]:
def obtener_antibiogramas(nombre_archivo):
    df = tabula.read_pdf(nombre_archivo, columns = [64, 218, 264, 296, 346, 373, 424, 450, 507], pages = 1, guess = False)[0]
    ya_hay_inicio_antibio = False

    for i in range(len(df)):
        contenido_linea = df.iloc[i].values
        if 'ANTIBIO' in contenido_linea:
            indice_inicio_antibio = i + 1
            ya_hay_inicio_antibio = True
        
        elif not(('ANTIBIO' in  contenido_linea) or ('Cepa 1' in contenido_linea) or ('Sensible' in contenido_linea) or ('Resistente' in contenido_linea) or ('Intermedio' in contenido_linea)) and (ya_hay_inicio_antibio):
            indice_termino_antibio = i
            break

    antibiograma_completo = df.iloc[indice_inicio_antibio: indice_termino_antibio].dropna(how = 'all', axis = 1)
    antibiograma_completo.columns = antibiograma_completo.iloc[0]
    antibiograma_completo = antibiograma_completo.iloc[1:, :]
    antibiograma_completo.set_index('ANTIBIOTICOS', inplace = True)

    tablas_cepas = []
    for i in range(int((antibiograma_completo.shape[1]) / 2)):
        df = antibiograma_completo.iloc[:, i * 2: (i * 2) + 2].dropna()
        tablas_cepas.append(df)

    return tablas_cepas


# if 'Sensible' or 


In [11]:
cepas = obtener_antibiogramas('00957643_CLAUDIA ANDREA BASCUÑAN SAAVEDRA.pdf')

In [12]:
cepas[0].index

22,Cepa 1,CIM
ANTIBIOTICOS,,
Amikacina,Sensible,<=8
Ceftazidima,Sensible,<=1
Ciprofloxacino,Sensible,"<=0,125"
Cefotaxima,Sensible,<=1
Ertapenem,Sensible,<=1
Gentamicina,Sensible,<=2
Imipenem,Sensible,"<=0,25"
Meropenem,Sensible,"<=0,5"
Piperacilina/Tazobactam,Sensible,<=4/4


In [7]:
def obtener_datos_demograficos_de_un_paciente(nombre_archivo):

    nombre_archivo = nombre_archivo[:-4] + '.pdf'

    with pdfplumber.open(nombre_archivo) as pdf:
        print(pdf.pages[0].extract_text())
        datos_personales_relevantes = pdf.pages[0].extract_text().split('\n')[3:12]

        nombre_paciente = datos_personales_relevantes[0].split(':')[1][:-10]
        n_orden = datos_personales_relevantes[0].split(':')[-1]
        rut = datos_personales_relevantes[1].split(':')[-1]

        linea_ingreso = datos_personales_relevantes[4].split(' ')
        try:
            fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d-%m-%Y %H:%M:%S')
        except ValueError:
            fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d/%m/%Y %H:%M:%S')


        linea_firma = datos_personales_relevantes[5].split(' ')
        try:
            fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d-%m-%Y %H:%M:%S')
        except ValueError:
            fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d/%m/%Y %H:%M:%S')

        seccion = datos_personales_relevantes[5].split(':')[1][:-13]
        tipo_muestra = datos_personales_relevantes[7].split(':')[-1]
        n_cultivo = datos_personales_relevantes[8].split(':', 1)[-1]
    
    if tipo_archivo == 'HONGOS':
            n_cultivo = tabla_cruda[tabla_cruda.iloc[:, 0] == 'Nº CULTIVO'].iloc[0, 4]
    
    return [fecha_ingreso, tipo_muestra, n_cultivo, rut, nombre_paciente, seccion, comentario, fecha_firma]
        
    comentario = None
    for filas in list(tabla_cruda.iloc[:, 0]):
        if type(filas) == str:
            if 'Avisado' in filas:
                comentario = 'ALERTA'
                print(f'El paciente {nombre_archivo} tiene una alerta!')

        
    
    return datos

In [8]:
def obtener_microorganismos_paciente(nombre_archivo):
    microorganismos = []
    with pdfplumber.open(nombre_archivo) as pdf:
        datos = pdf.pages[0].extract_text().split('\n')
        for linea in datos:
            if (('Cepa 1' in linea) or ('Cepa 2' in linea) or ('Cepa 3' in linea) or ('Cepa 4' in linea)) and ('ufc' in linea):
                linea_separada = linea.split(' ')
                nombre_microorganismo = linea_separada[2] + ' ' + linea_separada[3]
                microorganismos.append(nombre_microorganismo)
    
    return microorganismos

In [9]:
antibios = obtener_entradas_todos_los_pacientes()

INSTITUTO NACIONAL DEL TORAX
SERVICIO DE LABORATORIOS
Fono: 25754955, Anexo: 254955
Nombre :CLAUDIA ANDREA BASCUÑAN SAAVEDRA Nº Orden :00957643
Sexo :FEMENINO Rut :12.169.031-4
Edad :53 Años Nº Interno :
Fono Paciente :94036799 Tipo paciente :Ambulatorio
Medico :Sin especificar Ingreso :01-07-2022 09:30:01
Servicio :BRONCOSCOPIA Fecha Firma :12-07-2022 10:10:50
Localización :Sin especificar Sección :Microbiología
Diagnóstico : Tipo muestra :Lavado Bronquial
Nº de Cultivo :957643
CULTIVO CORRIENTE
TINCION DE GRAM
BACILOS GRAM - Muy escasos
CELULAS DESCAMATIVAS 0 - 1 X Campo
POLIMORFOS NUCLEARES 0 - 1 X Campo
CULTIVO CORRIENTE
MICROORGANISMOS RECUENTO
Cepa 1 Klebsiella pneumoniae Mas de 100.000 ufc/ml
Cepa 2 Pseudomonas aeruginosa Mas de 100.000 ufc/ml
Cepa 3 Pseudomonas aeruginosa Mas de 100.000 ufc/ml
ANTIBIOGRAMA
ANTIBIOTICOS Cepa 1 CIM Cepa 2 CIM Cepa 3 CIM
Amikacina Sensible <=8 Sensible <=8 Sensible .
Ceftazidima Sensible <=1 Sensible 8 Resistente .
Ciprofloxacino Sensible <=0,125 

NameError: name 'tipo_archivo' is not defined